In [1]:
!pip install yfinance


In [2]:
import yfinance as yf
import pandas as pd


nvda = yf.download("NVDA", start="2022-01-01", end="2025-08-28", interval='1d')

/tmp/ipython-input-4101073138.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  nvda = yf.download("NVDA", start="2022-01-01", end="2025-08-28", interval='1d')
[*********************100%***********************]  1 of 1 completed


In [3]:

if isinstance(nvda.columns, pd.MultiIndex):
    nvda.columns = [col[0] for col in nvda.columns]

nvda["Return"] = nvda["Close"].pct_change()
nvda["MA10"] = nvda["Close"].rolling(window=10).mean()
nvda["MA50"] = nvda["Close"].rolling(window=50).mean()
nvda["Volatility"] = nvda["Return"].rolling(window=10).std()
nvda = nvda.dropna()
print(nvda.head(15))
print("\nAvailable columns:", nvda.columns)


                Close       High        Low       Open     Volume    Return  \
Date                                                                          
2022-03-15  22.935062  22.999956  21.286790  21.482465  491996000  0.077028   
2022-03-16  24.455545  24.556378  23.133733  23.461190  671422000  0.066295   
2022-03-17  24.725100  24.800972  23.866519  24.080167  471941000  0.011022   
2022-03-18  26.409313  26.525123  24.583335  24.758045  730719000  0.068118   
2022-03-21  26.689848  27.107159  25.924114  26.463224  591727000  0.010623   
2022-03-22  26.480196  27.193018  26.028943  26.682860  547007000 -0.007855   
2022-03-23  25.591665  26.568049  25.532763  26.082851  502120000 -0.033555   
2022-03-24  28.103506  28.273225  25.864211  26.122784  877379000  0.098151   
2022-03-25  27.646271  28.311171  27.225966  27.839951  579016000 -0.016270   
2022-03-28  28.172400  28.203348  27.161072  27.710165  425494000  0.019031   
2022-03-29  28.608675  28.898195  27.933791  28.6476

In [4]:
nvda["Target_UpDown"] = (nvda["Close"].shift(-1) > nvda["Close"]).astype(int)
nvda["Target_Close"] = nvda["Close"].shift(-1)
nvda.dropna()

,Close,High,Low,Open,Volume,Return,MA10,MA50,Volatility,Target_UpDown,Target_Close
Date,,,,,,,,,,,
2022-03-15,22.935062,22.999956,21.286790,21.482465,491996000,0.077028,22.543809,24.608723,0.047368,1,24.455545
2022-03-16,24.455545,24.556378,23.133733,23.461190,671422000,0.066295,22.571363,24.496511,0.051139,1,24.725100
2022-03-17,24.725100,24.800972,23.866519,24.080167,471941000,0.011022,22.676390,24.406280,0.050523,1,26.409313
2022-03-18,26.409313,26.525123,24.583335,24.758045,730719000,0.068118,23.027509,24.383392,0.052080,1,26.689848
2022-03-21,26.689848,27.107159,25.924114,26.463224,591727000,0.010623,23.564820,24.354655,0.042995,0,26.480196
...,...,...,...,...,...,...,...,...,...,...,...
2025-08-20,175.399994,176.000000,168.800003,175.169998,215142700,-0.001366,180.579999,165.672800,0.013444,0,174.979996
2025-08-21,174.979996,176.899994,173.809998,174.850006,140040900,-0.002395,180.000998,166.293400,0.013008,1,177.990005
2025-08-22,177.990005,178.589996,171.199997,172.610001,172789400,0.017202,179.529999,166.996600,0.013912,1,179.809998


In [5]:
from sklearn.model_selection import train_test_split

X = nvda[["Return", "MA10", "MA50", "Volatility"]]
y = nvda[["Target_UpDown", "Target_Close"]]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

y_class_train = y_train["Target_UpDown"]
y_class_test  = y_test["Target_UpDown"]

y_reg_train   = y_train["Target_Close"]
y_reg_test    = y_test["Target_Close"]

In [6]:
print(nvda["Target_UpDown"].value_counts())


Target_UpDown
1    470
0    397
Name: count, dtype: int64


## THE CLASSIFICATION MODEL

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_class_train)


y_class_pred = clf.predict(X_test)


print("Classification Accuracy:", accuracy_score(y_class_test, y_class_pred))
print("\nClassification Report:\n", classification_report(y_class_test, y_class_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_class_test, y_class_pred))



Classification Accuracy: 0.5517241379310345

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        78
           1       0.55      1.00      0.71        96

    accuracy                           0.55       174
   macro avg       0.28      0.50      0.36       174
weighted avg       0.30      0.55      0.39       174


Confusion Matrix:
 [[ 0 78]
 [ 0 96]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


The model isn’t predicting well because the dataset and features don’t give it enough clear patterns to separate Up vs Down days. Since there are slightly more Up days (470 vs 397), the logistic regression model falls into a “lazy” solution: it always predicts Up to maximize accuracy without actually learning meaningful trends.

That’s why the confusion matrix is:

[[ 0 78]  
 [ 0 96]]


It shows the model never predicted a Down (0). In other words, it learned a trivial rule (“always Up”) instead of understanding the stock’s real movements.